In [48]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append('../src')

from data import tone_analyzer

%aimport data.tone_analyzer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [95]:
from dotenv import load_dotenv
load_dotenv()
import os
import requests
import time
import textrazor

In [15]:
from pymongo import MongoClient
import numpy as np

MONGO_PASSWORD = os.environ['MONGO_USER_PASSWORD']
CROWD_TANGLE = os.environ['CROWD_TANGLE_API_TOKEN']

config = {
  'host': '18.216.52.13:27017',
  'username': 'mongo_user',
  'password': MONGO_PASSWORD,
  'authSource': 'reddit_climate_news'
}

db = MongoClient(**config).reddit_climate_news

# Crowd Tangle

In [40]:
def get_crowd_tangle(link, ct_token):
    ct_inter_req = f'https://api.crowdtangle.com/ce/interactions?link={link}&token={ct_token}'
    inter_resp = requests.get(ct_inter_req)

    return inter_resp.json()

In [ ]:
count = 0
for record in db.climate_submissions.find():
    url = record['url']
    ct_res = get_crowd_tangle(url, CROWD_TANGLE)
    print(count)
    db.climate_submissions.update_one({'_id': record['_id']}, {'$set': {'crowd_tangle': ct_res}})
    count += 1

In [26]:
codes = np.array([x['crowd_tangle']['status'] for x in db.climate_submissions.find() if 'crowd_tangle' in x])

In [33]:
np.unique(codes, return_counts=True)

(array([200, 400, 401]), array([1571,   96,   16]))

textrazor.TextRazorAnalysisException

# Text Razor

In [ ]:
textrazor.api_key = os.environ['TEXT_RAZOR_TWO']

client = textrazor.TextRazor(extractors=["entities", "topics"])
client.set_cleanup_return_cleaned(True)

count = 0
error_count = 0

for record in db.climate_submissions.find({'text_razor': {'$exists': False}}):
    try:
        url = record['url']
        res = client.analyze_url(url)
        if count % 100 == 0:
            print(error_count)
            print(count)
        db.climate_submissions.update_one({'_id': record['_id']}, {'$set': {'text_razor': res.json}})
        count += 1
    except KeyboardInterrupt as e:
        raise e
    except textrazor.TextRazorAnalysisException as e:
        db.climate_submissions.update_one({'_id': record['_id']}, {'$set': {'text_razor': str(e)}})
        count += 1
        print(e)
        error_count += 1

In [92]:
db.climate_submissions.find({'text_razor': {'$exists': False}}).count()

/Users/willyraedy/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


2068

In [93]:
db.climate_submissions.find({'text_razor': {'$exists': True}}).count()

/Users/willyraedy/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


211

# Tone

In [49]:
count = 0
for record in db.climate_submissions.find():
    title = record['title']
    res = tone_analyzer.get(title)
    if count % 100 == 0: 
        print(count)
    db.climate_submissions.update_one({'_id': record['_id']}, {'$set': {'tone_analyzer': res}})
#     time.sleep(0.2)
    count += 1

0
100
200
300
400
500
600
700
800


InvalidDocument: cannot encode object: ConnectTimeout(MaxRetryError("HTTPSConnectionPool(host='iam.cloud.ibm.com', port=443): Max retries exceeded with url: /identity/token (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x1024371090>, 'Connection to iam.cloud.ibm.com timed out. (connect timeout=60)'))")), of type: <class 'requests.exceptions.ConnectTimeout'>

# Sentiment

In [35]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [37]:
analyzer = SentimentIntensityAnalyzer()

count = 0
for record in db.climate_submissions.find():
    title = record['title']
    res = analyzer.polarity_scores(title)['compound']
    if count % 100 == 0: 
        print(count)
    db.climate_submissions.update_one({'_id': record['_id']}, {'$set': {'sentiment_score': res}})
    count += 1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [50]:
db.climate_submissions.find_one()

{'_id': ObjectId('5e65d004bd71f87e9891c891'),
 'all_awardings': [],
 'allow_live_comments': False,
 'author': 'PBR--Streetgang',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_nssruw2',
 'author_patreon_flair': False,
 'author_premium': True,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1583573838,
 'domain': 'cnn.com',
 'full_link': 'https://www.reddit.com/r/worldnews/comments/fet0fd/these_south_african_farmers_say_their_cows_can/',
 'gildings': {},
 'id': 'fet0fd',
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': True,
 'is_self': False,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': False,
 'no_follow': True,
 'num_comments': 1,
 'num_cro